<a href="https://colab.research.google.com/github/Fr33zz/mw/blob/master/17%20megafon.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
#import matplotlib.pyplot as plt
#%matplotlib inline
#import seaborn as sns
#import math
from datetime import date
import datetime

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from google.colab import auth
auth.authenticate_user()
import gspread
from oauth2client.client import GoogleCredentials
gc = gspread.authorize(GoogleCredentials.get_application_default())

In [ ]:
orders = gc.open_by_url('https://docs.google.com/spreadsheets/d/106Mk7_n9iXmCB-fuPFgpWsV1t7JtoAdPaePi5x8BFTM')

payments = gc.open_by_url('https://docs.google.com/spreadsheets/d/1OKkFUxN-a55K52Z6QO0wnZg9SB-_6d3VZ_BXDCYsGYc')

closed = gc.open_by_url('https://docs.google.com/spreadsheets/d/1Gs7OkvQLNpt5IDTrsP5PBWbrSTO89zOPysbpbp7uzrQ')

sch_pay = gc.open_by_url('https://docs.google.com/spreadsheets/d/1AkH2q9AGbFuaXW3fUlrvvzcObFAr0vOkYdr2sNb4uLw')

opened = gc.open_by_url('https://docs.google.com/spreadsheets/d/1xUc4ayCcVNpkT1Bcx8_Hk84g02iM3U5GkxSSbZMMGAk')

charge = gc.open_by_url('https://docs.google.com/spreadsheets/d/1gOgXDjkxdjNaE-O-jprV77Tucdqm8uE-vg0NHDxP8xk')

def get_data(spreadsheet, list_number=0):
  ind = pd.DataFrame(spreadsheet.worksheets()[list_number].get_all_values())
  ind = ind.rename(columns=ind.iloc[0]).drop(index=0).reset_index().drop(columns='index')
  return ind

def get_titles(x, type_=1):
  '''type = {0:"list", 1:"df"}'''
  if type_==0:
    return [i.title for i in x.worksheets()]
  if type_==1:
    return pd.DataFrame([i.title for i in x.worksheets()])


def to_num(data, columns, print_=False, list_=False):
  d = data.copy()
  l = []
  for i in columns:
    try:       
      d[i] = d[i].map(lambda x: '.'.join(''.join(''.join(str(x).split('\xa0')).split(' ')).split(',')) if x!='' else np.nan).astype('float')
    except:
      l.append(i)
      if print_:
        print(print_,' : ', i)
      else:
        None
  if list_:
    return l, d
  else:
    return d


def to_dates(data, columns, print_=False, list_=False):
  d = data.copy()
  l = []
  for i in columns:
    try:
      
      d[i] = d[i].replace({'':np.nan})
      d[i] = d[i].str.split('T').map(lambda x: x[0] if x==x else x)
      #d[i] = d[i].str.split('-').map(lambda x: int(x[2]) if x==x else x)
      #d[i] = pd.to_datetime(d[i], format='%Y-%m-&d')
      d[i] = pd.to_datetime(d[i])
    
    except:
      l.append(i)
      if print_:
        print(print_, ' : ', i)
  if list_:
    return l, d
  else:
    return d

#for i in dates:
#  d[i] = pd.to_datetime(d[i].replace({'':np.nan}), format='%d.%m.%Y')

In [ ]:
#ord = get_data(orders, 0)

#clos = pd.concat([get_data(closed, i) for i in [0,2,4,5,6,7]])

#pay = get_data(payments, 0)
#legal = get_data(payments, 3)

#opn = get_data(opened, 0)
#loan = get_data(opened, 0)
#claim = get_data(opened, 1)

mw0 = pd.read_csv('/content/drive/My Drive/analytics/17 new scoring/data2scoring.csv')
at0 = pd.read_csv('/content/drive/My Drive/analytics/17 new scoring/data2scoring_AT.csv')

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (29,32,52,60,63,73,74,75,76,77,78,79,80,81,82,91,96,108,113,114,118,119,120,121,122,123,124,125) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (44) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [ ]:
#cold[0].str.split('_id').map(len).unique()
#cold[cold[0].str.split('amount').map(len)==2]

cold = pd.DataFrame(mw0.columns)
cold['mw'] = True
cold = pd.concat([cold, pd.DataFrame([at0.columns,
                                      np.full(at0.shape[1], True)],
                                     index=[0,'at']).T,])\
      .reset_index().drop(columns='index')

cold = cold.fillna(False)

dd = {'date':['_at', '_date', 'date_', '_dt'],
      'num':['amount', 'id'],
      'string':['state', 'phone', 'email', 'type', 'address', 'name',
                'passport', 'masked'],
      'json':['scoring_data', 'application_info', 'collected_data',
              'partner_request', 'payment_schedule_info', 'address'],
      'tables':['agreement', 'users', ]}

def set_type(data, names_col, type_dict):
  try:
    for i in type_dict:
      data[i] = False
      for j in type_dict[i]:
        data.loc[data[data[names_col].str.split(j).map(len)==2].index, i] = True
    return data
  except:
    print('smth went wrong')
    return -1

cold = set_type(cold, 0, dd)
cold['counter'] = cold.iloc[:,2:].sum(axis=1)
cold.loc[cold[cold['counter']==2].index,'string'] = False
cold['counter'] = cold.iloc[:,2:-1].sum(axis=1)

In [ ]:
mw0 = to_num(mw0, cold[cold['mw'] & cold['num']][0].values, 'mw')
at0 = to_num(at0, cold[cold['at'] & cold['num']][0].values, 'at')

mw  :  masked_credit_agreement_id
mw  :  masked_id
mw  :  middle_name
mw  :  middle_name.1
at  :  masked_id
at  :  middle_name


In [ ]:
mw0 = to_dates(mw0, cold[cold['mw']&cold['date']][0].values, 'mw')
at0 = to_dates(at0, cold[cold['at']&cold['date']][0].values, 'at')

mw  :  expires_at
mw  :  deleted_at
mw  :  birth_date
mw  :  issue_date
mw  :  imported_at.1
mw  :  deleted_at.1
mw  :  birth_date.1
mw  :  issue_date.1
mw  :  expiration_date
mw  :  imported_at.2
mw  :  deleted_at.2
mw  :  imported_at.3
mw  :  expiration_date.1
mw  :  deleted_at.3
mw  :  deleted_at.4
at  :  document_expiration_date


In [ ]:
at0['postal_code'] = at0['address'].str.split('postal_code').\
    map(lambda x: x[1][3:9] if x==x and len(x)==2 else np.nan)

ind = at0[at0['phone'].isna()].index
at0.loc[ind, 'phone'] = at0.loc[ind, 'phone.1']

ind = at0[at0['birthday'].isna()].index
at0.loc[ind, 'birthday'] = at0.loc[ind, 'date_of_birth']

ind = at0[at0['building'].isna()].index
at0.loc[ind, 'building'] = at0.loc[ind, 'house_number']

at0['phone'] = at0['phone'].map(lambda x: str(int(x)) if x==x else np.nan)
mw0['postcode'] = mw0['postcode'].map(lambda x: str(int(x)) if x==x else np.nan)

mw0['DISBURSED'] = mw0['agr_state'].notna().map(lambda x: 1 if x else 0)
at0['DISBURSED'] = at0['state'].isin(['closed', 'open']).map(lambda x: 1 if x else 0)

mw0['AMOUNT'] = mw0['initial_payment_amount']*5/2
at0['AMOUNT'] = at0['first_payment_amount']*5/2

at0['SOCIAL_DEFAULT'] = (at0['state']=='open').map(lambda x: 1 if x else 0)
mw0['SOCIAL_DEFAULT'] = (mw0['agr_state']=='open')&(mw0['created_at'] < datetime.datetime.today() - datetime.timedelta(60))
mw0['SOCIAL_DEFAULT'] = mw0['SOCIAL_DEFAULT'].map(lambda x: 1 if x else 0)

mw0['FRAUD_DEFAULT'] = mw0['SOCIAL_DEFAULT']*((datetime.datetime.today() - mw0['created_at']).dt.days > 365)
at0['FRAUD_DEFAULT'] = at0['SOCIAL_DEFAULT']*((datetime.datetime.today() - at0['created_at']).dt.days > 365)

In [ ]:
at0['first_payment_amount'].map(type).unique()

array([<class 'float'>], dtype=object)

In [ ]:
cmw = [i for i in 'after4, state, agr_state, initial_payment_amount, masked_credit_agreement_id, \
phone, created_at, closed_at, paused_at, birth_date, sex, city, street, building, \
housing, apartment, postcode, partner_id, DISBURSED'.split(', ')]

cat = [i for i in 'after4, state, created_at, masked_id, first_payment_amount, \
phone, birthday, postal_code, apartment, city, street, \
house_number, sex, DISBURSED'.split(', ')]

In [ ]:
cmw = {#'after4': '',
 'masked_credit_agreement_id': 'ID',
 'created_at': 'DATE_APP',
 'phone': 'MOB_NUM',
 'AMOUNT':'AMOUNT',
 'DISBURSED':'DISBURSED',
 'SOCIAL_DEFAULT':'SOCIAL_DEFAULT',
 'FRAUD_DEFAULT':'FRAUD_DEFAULT',
 'partner_id': 'CHANNEL_ID',
 'postcode': 'HOME_ZIP',
 'building': 'HOME_HOUSE',
 'street': 'HOME_STREET',
 'city': 'HOME_CITY',
 'birth_date': 'DATE_OF_BIRTH'}



cat = {#'after4': '',
 'masked_id': 'ID',
 'created_at': 'DATE_APP',
 'phone': 'MOB_NUM',
 'AMOUNT':'AMOUNT',
 'DISBURSED':'DISBURSED',
 'SOCIAL_DEFAULT':'SOCIAL_DEFAULT',
 'FRAUD_DEFAULT':'FRAUD_DEFAULT',
 'postal_code': 'HOME_ZIP',
 'house_number': 'HOME_HOUSE',
 'street': 'HOME_STREET',
 'city': 'HOME_CITY',
 'birthday': 'DATE_OF_BIRTH'}


In [ ]:
path = '/content/drive/My Drive/analytics/17 new scoring/data2megafon.csv'

pd.concat([mw0[mw0['after4']][cmw.keys()].rename(columns=cmw),
           at0[at0['after4']][cat.keys()].rename(columns=cat)])\
    .reset_index().drop(columns=['index']).to_csv(path)

In [ ]:
mw0[mw0['after4']][cmw.keys()].rename(columns=cmw)

,ID,DATE_APP,MOB_NUM,AMOUNT,DISBURSED,SOCIAL_DEFAULT,FRAUD_DEFAULT,CHANNEL_ID,HOME_ZIP,HOME_HOUSE,HOME_STREET,HOME_CITY,DATE_OF_BIRTH
1,MW333333335,2017-04-13,79175524108,3007.5,0,0,0,2.0,125315,23/5,Усиевича,Москва,1986-06-01
2,MW333333337,2017-04-14,79175524108,3012.5,0,0,0,2.0,125319,23/5,Усиевича,Москва,1986-06-01
3,MW333333339,2017-04-14,79175524108,3227.5,0,0,0,2.0,125319,23/5,Усиевича,Москва,1986-06-01
4,MW333333331,2017-04-17,79262314278,5717.5,0,0,0,2.0,111116,4,Авиамоторная,Москва,1977-10-21
5,MW333333332,2017-04-17,79999999999,8947.5,0,0,0,2.0,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
61950,MW333395363,2020-11-08,79867880830,4655.0,1,0,0,27.0,461047,22,микрорайон 7а,город Бузулук,1998-09-02
61951,MW333395364,2020-11-08,79819517328,15255.0,0,0,0,10.0,NaN,NaN,NaN,NaN,NaN
61952,MW333395365,2020-11-08,79009913138,5735.0,0,0,0,14.0,NaN,NaN,NaN,NaN,NaN
61953,MW333395366,2020-11-08,79168684557,20125.0,0,0,0,10.0,428038,35,бульвар А.Миттова,город Чебоксары,1983-06-04
